# SOEN_PROJECT

In [3]:
# Importing libraries and setting the path.

import os
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession


warnings.filterwarnings('ignore')

path = 'archive/A_DeviceMotion_data/A_DeviceMotion_data/'

## Reading of files 

In [4]:
# Setting up lists for importing the data.

list_of_folders = os.listdir(path)
list_of_directories = [path + i for i in list_of_folders]
list_of_pdfs = []


activities = ['dws', 'ups', 'sit', 'std', 'wlk', 'jog']

spark = SparkSession.builder.appName('SOEN_PROJECT_').getOrCreate()

for i in list_of_directories:
           
    if ".DS_Store" in i:
        continue
    
    list_of_csv = os.listdir(i)
    
    for j in list_of_csv:
        
        k = ps.read_csv(i + '/' + j, sep=',', header="infer").rename(columns={"_c0":"time"})
        
        k['subject'] = int(j.split('_')[1].split('.')[0])
        k['activity'] = i.split('/')[-1].split('_')[0]
        k['trial'] = int(i.split('/')[-1].split('_')[1])
        k.insert(0, 'accel_x', k['userAcceleration.x'] + k['gravity.x'])
        k.insert(1, 'accel_y', k['userAcceleration.y'] + k['gravity.y'])
        k.insert(2, 'accel_z', k['userAcceleration.z'] + k['gravity.z'])
        k.insert(3, 'accel_norm', np.sqrt(k['accel_x']**2 + k['accel_y']**2 + k['accel_z']**2))
        list_of_pdfs.append(k)


In [5]:
pdf = ps.concat(list_of_pdfs).set_index('time')
pdf = pdf.drop(['attitude.roll', 'attitude.pitch', 'attitude.yaw', 'gravity.x', 'gravity.y', 'gravity.z', 'userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z'], axis=1)
pdf = pdf.rename(columns={'rotationRate.x': 'gyro_x', 'rotationRate.y': 'gyro_y', 'rotationRate.z': 'gyro_z'})

In [6]:
pdf.head(20)

23/03/03 14:32:39 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB


23/03/03 14:32:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/dws_11/sub_12.csv


,accel_x,accel_y,accel_z,accel_norm,gyro_x,gyro_y,gyro_z,subject,activity,trial
time,,,,,,,,,,
0,-0.122085,0.626434,-0.161957,0.658448,-1.264215,-1.027909,-0.947909,12,dws,11
1,-0.160369,0.946457,-0.288605,1.002393,-1.162024,-0.269118,-0.848823,12,dws,11
2,-0.178482,1.202301,-0.178314,1.228487,-0.665042,0.520170,-0.726722,12,dws,11
3,-0.033493,1.307663,-0.223786,1.327096,-0.079809,0.055322,-0.604534,12,dws,11
4,-0.141067,1.322677,-0.641449,1.476764,0.456097,-0.186877,-0.441315,12,dws,11
5,-0.020950,1.711899,-0.444931,1.768898,1.531273,-0.369080,-0.563895,12,dws,11
6,0.137573,1.897262,-0.248901,1.918458,2.413919,-0.488344,-0.275566,12,dws,11
7,0.136551,1.647385,-0.086471,1.655295,3.309900,-0.802186,0.175475,12,dws,11
8,-0.477051,1.472824,-0.042968,1.548753,3.873452,-0.347702,0.582617,12,dws,11
